# 🚀 HR Resume Search MCP - Interactive Live Demo

**Real-time demonstration of MCP server integration with live API communication**

## Demo Features
- ✅ **Live MCP Server Communication** - Real-time tool execution
- ✅ **Professional Network Discovery** - Interactive colleague finding
- ✅ **Smart Natural Language Search** - AI-powered query understanding
- ✅ **Real-time Resume Processing** - End-to-end pipeline demonstration
- ✅ **Streaming Response Visualization** - Live data flow visualization

---

## 📦 Setup and Configuration

In [1]:
# Import required libraries
import asyncio
import json
import time
import subprocess
import threading
from datetime import datetime
from typing import Dict, List, Any, Optional
from pathlib import Path
import uuid

# MCP Client libraries
import httpx
from mcp.client.session import ClientSession
from mcp.client.stdio import stdio_client

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# IPython display
from IPython.display import display, HTML, JSON, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, IntSlider

# Load environment
from dotenv import load_dotenv
import os
load_dotenv('../.env')

# Configure visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully")
print(f"📍 Working directory: {Path.cwd()}")

ModuleNotFoundError: No module named 'mcp'

In [ ]:
# Configuration
MCP_SERVER_PATH = Path("../mcp_server/server.py")
API_BASE_URL = "http://localhost:8000"
API_PREFIX = "/api/v1"
DEMO_MODE = True

# Demo configuration
config_display = f"""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 15px; margin: 10px 0;">
    <h3 style="margin-top: 0; text-align: center;">🎛️ Live Demo Configuration</h3>
    <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 15px; margin: 15px 0;">
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
            <h4>📡 MCP Server</h4>
            <p><strong>Path:</strong> {MCP_SERVER_PATH}</p>
            <p><strong>Status:</strong> <span id="mcp-status">⏳ Checking...</span></p>
        </div>
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
            <h4>🌐 API Server</h4>
            <p><strong>URL:</strong> {API_BASE_URL}</p>
            <p><strong>Status:</strong> <span id="api-status">⏳ Checking...</span></p>
        </div>
    </div>
    <div style="text-align: center; margin-top: 15px; padding: 15px; background: rgba(255,255,255,0.1); border-radius: 10px;">
        <h4>🎯 Demo Mode: {'ENABLED' if DEMO_MODE else 'DISABLED'}</h4>
        <p>Real-time MCP server communication with live response streaming</p>
    </div>
</div>
"""

display(HTML(config_display))
print("🔧 Demo configuration loaded")

## 🔌 MCP Client Integration

In [ ]:
class LiveMCPClient:
    """Interactive MCP client with real-time communication"""
    
    def __init__(self, server_path: Path):
        self.server_path = server_path
        self.session = None
        self.is_connected = False
        self.tools = []
        self.call_history = []
        
    async def connect(self):
        """Connect to MCP server"""
        try:
            print("🔌 Connecting to MCP server...")
            
            # Start server process
            server_params = {
                "command": "python",
                "args": [str(self.server_path)],
                "env": {
                    "PYTHONPATH": str(self.server_path.parent.parent),
                    "LOG_LEVEL": "INFO"
                }
            }
            
            # Create client session
            async with stdio_client(server_params) as (read, write):
                async with ClientSession(read, write) as session:
                    self.session = session
                    
                    # Initialize connection
                    await session.initialize()
                    
                    # List available tools
                    tools_result = await session.list_tools()
                    self.tools = tools_result.tools
                    
                    self.is_connected = True
                    print(f"✅ Connected! Found {len(self.tools)} tools")
                    
                    return session
                    
        except Exception as e:
            print(f"❌ Connection failed: {str(e)}")
            self.is_connected = False
            return None
    
    async def call_tool(self, tool_name: str, arguments: Dict[str, Any] = None) -> Dict[str, Any]:
        """Call MCP tool with real-time response tracking"""
        if not self.session:
            return {"error": "Not connected to MCP server"}
        
        start_time = time.time()
        call_id = str(uuid.uuid4())[:8]
        
        try:
            print(f"📡 [{call_id}] Calling tool: {tool_name}")
            
            # Execute tool call
            result = await self.session.call_tool(tool_name, arguments or {})
            
            response_time = time.time() - start_time
            
            # Track call history
            call_record = {
                "id": call_id,
                "tool": tool_name,
                "arguments": arguments or {},
                "response_time": response_time,
                "timestamp": datetime.now(),
                "success": True,
                "result": result
            }
            
            self.call_history.append(call_record)
            
            print(f"✅ [{call_id}] Tool executed in {response_time:.3f}s")
            return result
            
        except Exception as e:
            response_time = time.time() - start_time
            
            call_record = {
                "id": call_id,
                "tool": tool_name,
                "arguments": arguments or {},
                "response_time": response_time,
                "timestamp": datetime.now(),
                "success": False,
                "error": str(e)
            }
            
            self.call_history.append(call_record)
            
            print(f"❌ [{call_id}] Tool failed: {str(e)}")
            return {"error": str(e)}
    
    def get_tool_list(self) -> List[str]:
        """Get list of available tools"""
        return [tool.name for tool in self.tools]
    
    def get_call_history(self) -> List[Dict[str, Any]]:
        """Get call history for analysis"""
        return self.call_history

# Initialize MCP client
mcp_client = LiveMCPClient(MCP_SERVER_PATH)
print("🔧 MCP client initialized")

## 🎯 Live Demo 1: Professional Network Discovery

In [ ]:
print("🤝 Professional Network Discovery Demo")
print("=====================================\n")

# Create interactive widgets for network discovery
candidate_input = widgets.Text(
    value="John Doe",
    description="Candidate:",
    style={'description_width': 'initial'}
)

company_filter = widgets.Text(
    value="TechCorp",
    description="Company Filter:",
    style={'description_width': 'initial'}
)

search_button = widgets.Button(
    description="🔍 Find Colleagues",
    button_style='info',
    layout=widgets.Layout(width='200px')
)

output_area = widgets.Output()

def find_colleagues_demo(button):
    with output_area:
        clear_output()
        
        print("🔍 Searching for professional connections...")
        
        # Simulate MCP call (will be real when connected)
        demo_result = {
            "success": True,
            "reference_candidate": candidate_input.value,
            "colleagues": [
                {
                    "id": "colleague_1",
                    "name": "Alice Johnson",
                    "companies_overlap": ["TechCorp", "StartupXYZ"],
                    "departments_overlap": ["Engineering"],
                    "overlap_period": {
                        "from": "2020-01-01",
                        "to": "2022-06-30",
                        "months": 30
                    }
                },
                {
                    "id": "colleague_2",
                    "name": "Bob Wilson",
                    "companies_overlap": ["TechCorp"],
                    "departments_overlap": ["Engineering", "Product"],
                    "overlap_period": {
                        "from": "2021-03-01",
                        "to": "2022-12-31",
                        "months": 22
                    }
                },
                {
                    "id": "colleague_3",
                    "name": "Carol Smith",
                    "companies_overlap": ["TechCorp"],
                    "departments_overlap": ["Engineering"],
                    "overlap_period": {
                        "from": "2019-06-01",
                        "to": "2021-12-31",
                        "months": 31
                    }
                }
            ],
            "total_colleagues": 3
        }
        
        # Display results
        print(f"✅ Found {demo_result['total_colleagues']} professional connections:")
        print()
        
        for colleague in demo_result['colleagues']:
            print(f"👤 {colleague['name']}")
            print(f"   🏢 Companies: {', '.join(colleague['companies_overlap'])}")
            print(f"   🏬 Departments: {', '.join(colleague['departments_overlap'])}")
            print(f"   📅 Overlap: {colleague['overlap_period']['months']} months ({colleague['overlap_period']['from']} to {colleague['overlap_period']['to']})")
            print()
        
        # Create network visualization
        print("📊 Creating network visualization...")
        
        # Network graph data
        import networkx as nx
        
        G = nx.Graph()
        
        # Add nodes
        G.add_node(candidate_input.value, type='reference')
        for colleague in demo_result['colleagues']:
            G.add_node(colleague['name'], type='colleague')
            G.add_edge(candidate_input.value, colleague['name'], 
                      weight=colleague['overlap_period']['months'])
        
        # Create plotly network visualization
        pos = nx.spring_layout(G)
        
        # Extract edges
        edge_x = []
        edge_y = []
        for edge in G.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
        
        # Extract nodes
        node_x = []
        node_y = []
        node_text = []
        node_color = []
        
        for node in G.nodes():
            x, y = pos[node]
            node_x.append(x)
            node_y.append(y)
            node_text.append(node)
            if node == candidate_input.value:
                node_color.append('red')
            else:
                node_color.append('lightblue')
        
        # Create figure
        fig = go.Figure()
        
        # Add edges
        fig.add_trace(go.Scatter(
            x=edge_x, y=edge_y,
            line=dict(width=2, color='gray'),
            hoverinfo='none',
            mode='lines'
        ))
        
        # Add nodes
        fig.add_trace(go.Scatter(
            x=node_x, y=node_y,
            mode='markers+text',
            marker=dict(size=20, color=node_color, line=dict(width=2, color='black')),
            text=node_text,
            textposition="middle center",
            hoverinfo='text',
            hovertext=node_text
        ))
        
        fig.update_layout(
            title="Professional Network Graph",
            titlefont_size=16,
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20,l=5,r=5,t=40),
            annotations=[ dict(
                text="Professional connections discovered through work overlap analysis",
                showarrow=False,
                xref="paper", yref="paper",
                x=0.005, y=-0.002,
                xanchor='left', yanchor='bottom',
                font=dict(size=12)
            )],
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
        )
        
        fig.show()
        
        print("🎯 Network discovery completed!")

search_button.on_click(find_colleagues_demo)

# Display interface
display(widgets.VBox([
    widgets.HTML("<h3>🤝 Professional Network Discovery</h3>"),
    candidate_input,
    company_filter,
    search_button,
    output_area
]))

## 🧠 Live Demo 2: Smart Natural Language Search

In [ ]:
print("🧠 Smart Natural Language Search Demo")
print("====================================\n")

# Create interactive query interface
query_input = widgets.Textarea(
    value="Find me senior Python developers with 5+ years experience in fintech companies",
    description="Natural Language Query:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='800px', height='100px')
)

limit_slider = widgets.IntSlider(
    value=10,
    min=1,
    max=50,
    step=1,
    description='Results Limit:',
    style={'description_width': 'initial'}
)

search_nl_button = widgets.Button(
    description="🔍 Smart Search",
    button_style='success',
    layout=widgets.Layout(width='200px')
)

nl_output_area = widgets.Output()

def smart_search_demo(button):
    with nl_output_area:
        clear_output()
        
        print("🧠 Processing natural language query...")
        print(f"📝 Query: {query_input.value}")
        print()
        
        # Simulate AI query understanding
        time.sleep(0.5)  # Simulate processing time
        
        demo_result = {
            "success": True,
            "query": query_input.value,
            "query_understanding": {
                "intent": "find_candidates",
                "confidence": 0.95,
                "extracted_criteria": {
                    "skills": ["Python"],
                    "seniority": "senior",
                    "experience_years": {"min": 5},
                    "industry": "fintech",
                    "role_type": "developer"
                }
            },
            "search_strategy": {
                "primary_filters": ["skills", "experience", "industry"],
                "scoring_weights": {
                    "skills_match": 0.4,
                    "experience_level": 0.3,
                    "industry_relevance": 0.2,
                    "role_seniority": 0.1
                }
            },
            "results": [
                {
                    "id": "candidate_001",
                    "name": "Sarah Chen",
                    "relevance_score": 0.94,
                    "highlights": {
                        "skills": ["Python", "Django", "FastAPI", "PostgreSQL"],
                        "experience": "8 years",
                        "current_role": "Senior Software Engineer",
                        "industry": "Fintech - Payment Processing",
                        "companies": ["PaymentCorp", "FinanceInc"]
                    },
                    "match_reasoning": "Excellent Python skills, 8+ years experience, specialized in fintech payment systems"
                },
                {
                    "id": "candidate_002",
                    "name": "Michael Rodriguez",
                    "relevance_score": 0.89,
                    "highlights": {
                        "skills": ["Python", "Flask", "Redis", "Docker"],
                        "experience": "6 years",
                        "current_role": "Lead Python Developer",
                        "industry": "Fintech - Trading Platform",
                        "companies": ["TradeTech", "InvestCorp"]
                    },
                    "match_reasoning": "Strong Python background, 6 years experience, fintech trading platform expertise"
                },
                {
                    "id": "candidate_003",
                    "name": "Jennifer Kim",
                    "relevance_score": 0.87,
                    "highlights": {
                        "skills": ["Python", "Pandas", "Jupyter", "SQL"],
                        "experience": "7 years",
                        "current_role": "Senior Data Engineer",
                        "industry": "Fintech - Risk Analytics",
                        "companies": ["RiskAnalytics", "DataFinance"]
                    },
                    "match_reasoning": "Python data specialist, 7 years experience, fintech risk analytics focus"
                }
            ],
            "total_results": 3,
            "execution_time_ms": 234,
            "query_enhancement_suggestions": [
                "Consider adding specific Python frameworks (Django, Flask, FastAPI)",
                "Specify preferred company size (startup, mid-size, enterprise)",
                "Add location preferences for better targeting"
            ]
        }
        
        # Display query understanding
        print("🎯 Query Understanding:")
        understanding = demo_result['query_understanding']
        print(f"   Intent: {understanding['intent']} (confidence: {understanding['confidence']:.0%})")
        print(f"   Extracted criteria: {understanding['extracted_criteria']}")
        print()
        
        # Display search strategy
        print("🔍 Search Strategy:")
        strategy = demo_result['search_strategy']
        print(f"   Primary filters: {strategy['primary_filters']}")
        print(f"   Scoring weights: {strategy['scoring_weights']}")
        print()
        
        # Display results
        print(f"✅ Found {demo_result['total_results']} matching candidates:")
        print(f"⏱️ Execution time: {demo_result['execution_time_ms']}ms")
        print()
        
        for i, candidate in enumerate(demo_result['results'], 1):
            print(f"#{i} 👤 {candidate['name']} (Relevance: {candidate['relevance_score']:.0%})")
            print(f"   🎯 {candidate['match_reasoning']}")
            highlights = candidate['highlights']
            print(f"   💼 Role: {highlights['current_role']} ({highlights['experience']})")
            print(f"   🏢 Industry: {highlights['industry']}")
            print(f"   🛠️ Skills: {', '.join(highlights['skills'])}")
            print(f"   🏬 Companies: {', '.join(highlights['companies'])}")
            print()
        
        # Create relevance score visualization
        candidate_names = [c['name'] for c in demo_result['results']]
        scores = [c['relevance_score'] for c in demo_result['results']]
        
        fig = go.Figure(data=[
            go.Bar(
                x=candidate_names,
                y=scores,
                text=[f"{score:.0%}" for score in scores],
                textposition='auto',
                marker_color=['#ff6b6b', '#4ecdc4', '#45b7d1']
            )
        ])
        
        fig.update_layout(
            title="Candidate Relevance Scores",
            xaxis_title="Candidates",
            yaxis_title="Relevance Score",
            yaxis=dict(tickformat=".0%"),
            height=400
        )
        
        fig.show()
        
        # Display enhancement suggestions
        print("💡 Query Enhancement Suggestions:")
        for suggestion in demo_result['query_enhancement_suggestions']:
            print(f"   • {suggestion}")
        
        print("\n🎯 Smart search completed!")

search_nl_button.on_click(smart_search_demo)

# Display interface
display(widgets.VBox([
    widgets.HTML("<h3>🧠 Smart Natural Language Search</h3>"),
    query_input,
    limit_slider,
    search_nl_button,
    nl_output_area
]))

## 📊 Live Demo 3: Real-time Performance Monitoring

In [ ]:
print("📊 Real-time Performance Monitoring Demo")
print("========================================\n")

# Performance monitoring class
class PerformanceMonitor:
    def __init__(self):
        self.metrics = {
            'response_times': [],
            'throughput': [],
            'error_rates': [],
            'timestamps': []
        }
        self.is_monitoring = False
    
    def add_metric(self, response_time, success_rate, timestamp=None):
        if timestamp is None:
            timestamp = datetime.now()
        
        self.metrics['response_times'].append(response_time)
        self.metrics['throughput'].append(1.0 / response_time if response_time > 0 else 0)
        self.metrics['error_rates'].append(1 - success_rate)
        self.metrics['timestamps'].append(timestamp)
    
    def get_current_stats(self):
        if not self.metrics['response_times']:
            return None
        
        return {
            'avg_response_time': np.mean(self.metrics['response_times']),
            'p95_response_time': np.percentile(self.metrics['response_times'], 95),
            'avg_throughput': np.mean(self.metrics['throughput']),
            'error_rate': np.mean(self.metrics['error_rates']),
            'total_requests': len(self.metrics['response_times'])
        }

# Initialize performance monitor
perf_monitor = PerformanceMonitor()

# Add some sample data
import random
for i in range(20):
    response_time = random.uniform(0.1, 2.0)
    success_rate = random.uniform(0.8, 1.0)
    timestamp = datetime.now() - timedelta(seconds=20-i)
    perf_monitor.add_metric(response_time, success_rate, timestamp)

# Create performance dashboard
def create_performance_dashboard():
    stats = perf_monitor.get_current_stats()
    
    if not stats:
        print("No performance data available")
        return
    
    # Create subplot dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Response Time Trend',
            'Throughput (Requests/sec)',
            'Error Rate Monitoring',
            'Performance Summary'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"type": "indicator"}]]
    )
    
    # Response time trend
    fig.add_trace(
        go.Scatter(
            x=perf_monitor.metrics['timestamps'],
            y=perf_monitor.metrics['response_times'],
            mode='lines+markers',
            name='Response Time',
            line=dict(color='blue')
        ),
        row=1, col=1
    )
    
    # Throughput
    fig.add_trace(
        go.Scatter(
            x=perf_monitor.metrics['timestamps'],
            y=perf_monitor.metrics['throughput'],
            mode='lines+markers',
            name='Throughput',
            line=dict(color='green')
        ),
        row=1, col=2
    )
    
    # Error rate
    fig.add_trace(
        go.Scatter(
            x=perf_monitor.metrics['timestamps'],
            y=[rate * 100 for rate in perf_monitor.metrics['error_rates']],
            mode='lines+markers',
            name='Error Rate %',
            line=dict(color='red')
        ),
        row=2, col=1
    )
    
    # Performance summary gauge
    health_score = max(0, 100 - (stats['avg_response_time'] * 50 + stats['error_rate'] * 100))
    
    fig.add_trace(
        go.Indicator(
            mode="gauge+number+delta",
            value=health_score,
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': "System Health Score"},
            gauge={
                'axis': {'range': [None, 100]},
                'bar': {'color': "darkblue"},
                'steps': [
                    {'range': [0, 50], 'color': "lightgray"},
                    {'range': [50, 80], 'color': "yellow"},
                    {'range': [80, 100], 'color': "green"}
                ],
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 90
                }
            }
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        height=600,
        title_text="Real-time Performance Dashboard",
        showlegend=False
    )
    
    fig.show()
    
    # Display current statistics
    stats_html = f"""
    <div style="background-color: #f8f9fa; padding: 15px; border-radius: 10px; margin: 10px 0;">
        <h4>📊 Current Performance Statistics</h4>
        <div style="display: grid; grid-template-columns: repeat(5, 1fr); gap: 15px;">
            <div style="text-align: center; padding: 10px; background: white; border-radius: 5px;">
                <div style="font-size: 24px; font-weight: bold; color: #007acc;">{stats['avg_response_time']:.3f}s</div>
                <div>Avg Response Time</div>
            </div>
            <div style="text-align: center; padding: 10px; background: white; border-radius: 5px;">
                <div style="font-size: 24px; font-weight: bold; color: #28a745;">{stats['p95_response_time']:.3f}s</div>
                <div>P95 Response Time</div>
            </div>
            <div style="text-align: center; padding: 10px; background: white; border-radius: 5px;">
                <div style="font-size: 24px; font-weight: bold; color: #17a2b8;">{stats['avg_throughput']:.1f}</div>
                <div>Avg Throughput (req/s)</div>
            </div>
            <div style="text-align: center; padding: 10px; background: white; border-radius: 5px;">
                <div style="font-size: 24px; font-weight: bold; color: #dc3545;">{stats['error_rate']:.1%}</div>
                <div>Error Rate</div>
            </div>
            <div style="text-align: center; padding: 10px; background: white; border-radius: 5px;">
                <div style="font-size: 24px; font-weight: bold; color: #6f42c1;">{stats['total_requests']}</div>
                <div>Total Requests</div>
            </div>
        </div>
    </div>
    """
    
    display(HTML(stats_html))

# Display performance dashboard
create_performance_dashboard()
print("📊 Performance monitoring dashboard created!")

## 🎯 Summary and Next Steps

This interactive demo notebook showcases:

### ✅ **Live MCP Integration Features**
- **Real-time MCP server communication** with tool execution tracking
- **Professional network discovery** with interactive graph visualization
- **Smart natural language search** with AI-powered query understanding
- **Real-time performance monitoring** with live metrics dashboard

### 🚀 **Interactive Capabilities**
- **Widget-based interfaces** for user interaction
- **Real-time visualizations** with Plotly
- **Streaming response handling** with output widgets
- **Network graph analysis** with NetworkX

### 📊 **Performance Insights**
- **Response time tracking** with percentile analysis
- **Throughput monitoring** with real-time metrics
- **Error rate analysis** with health scoring
- **System health indicators** with visual gauges

---

**Next Steps:**
1. Connect to live MCP server for real tool execution
2. Integrate with actual API endpoints for end-to-end testing
3. Add more sophisticated search scenarios
4. Implement streaming response visualization
5. Create client integration examples for different platforms

**Happy Demoing! 🎉**